In [2]:
from google.colab import files
files.upload()

Saving data.csv to data.csv


{'data.csv': b',title,response\n0,hi what is your name?,hi this is Sofia.\n1,nice to meet you!,nice to meet you too!\n2,which programming language do you use?,i like python.\n3,where do you live?,i live in gwalior.\n4,what is your major?,i study software engineering.\n5,what do you want to drink?,beer please!\n6,what is your favourite beer?,leffe brown!\n7,see you later.,bye bye.\n8,thank you,welcome\n9,what is your occupation?,software engineer\n10,who is your father?,chinmay\n11,what can you do?,i am here to help you.\n12,i am depressed,why?\n13,my boss is harassing me,"okay, gotcha"\n14,what should i do?,try watching recommended video on this site\n15,what can i do?,try watching recommended video on this site\n16,i want to kill myself,don\xe2\x80\x99t life is beautiful.\n17,my friend fooled me,don\xe2\x80\x99t be sad.\n18,my girlfriend fooled me,don\xe2\x80\x99t be sad.\n19,my gf fooled me,don\xe2\x80\x99t be sad. it\xe2\x80\x99s okay\n20,my gf backstabbed me,i understand\n21,my gf 

In [3]:
import collections

import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding, CuDNNLSTM, GlobalMaxPooling1D, GlobalAveragePooling1D, CuDNNGRU
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import helper
import json
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import re

Using TensorFlow backend.


In [0]:
# df = pd.read_csv('scrap_context.csv')
# df_tp = pd.read_csv('reddit.txt', sep="_eou_",error_bad_lines=False, header=['title','response'] )
# df_tp.head()

In [0]:
# df.shape
# df_tp.shape

In [0]:
# df_tp.iloc[0]['text']

In [0]:
# df = pd.DataFrame(columns=['title','response'])
# df

In [0]:
df = pd.DataFrame(columns=['title','response'])

In [0]:
# df_1 = pd.read_csv('general2.txt', sep='_eou_')
# df_1.head()

In [0]:
# df = pd.DataFrame(columns=['title','response'])
# for i in tqdm(range(1,len(df_tp)-1)):
#   if 1 <= len(df_tp.iloc[i]['text'].split()) <= 7:
#     if 1 <= len(df_tp.iloc[i+1]['text'].split()) <= 7:
#       title = df_tp.iloc[i]['text']
#       response = df_tp.iloc[i+1]['text']
#       df = df.append({'title':title, 'response':response}, ignore_index=True)

In [0]:
# df.shape

In [0]:
# df.head()

In [0]:
# df_gen = pd.read_csv('general.csv', names=['title','response'])
# df_gen.shape

In [0]:
# df_gen.head()

In [0]:
# df_gen2 = pd.read_csv('general2.csv', names=['title','response'])
# df_gen2.shape

(500, 2)

In [0]:
# df_gen2.head()

In [0]:
df = pd.read_csv('data.csv', index_col=0)

In [6]:
df.head()

,title,response
0,hi what is your name?,hi this is Sofia.
1,nice to meet you!,nice to meet you too!
2,which programming language do you use?,i like python.
3,where do you live?,i live in gwalior.
4,what is your major?,i study software engineering.


In [0]:
# Append the data of both the csv files to one dataframe.
# for i in tqdm(range(0,len(df_gen)-1)):
#   title = df_gen.iloc[i]['title']
#   response = df_gen.iloc[i]['response']
#   df = df.append({'title':title, 'response':response}, ignore_index=True)

100%|██████████| 199/199 [00:00<00:00, 385.37it/s]


In [0]:
# for i in tqdm(range(0,len(df_gen2)-1)):
#   title = df_gen2.iloc[i]['title']
#   response = df_gen2.iloc[i]['response']
#   df = df.append({'title':title, 'response':response}, ignore_index=True)

100%|██████████| 499/499 [00:01<00:00, 381.37it/s]


In [7]:
df.head()

,title,response
0,hi what is your name?,hi this is Sofia.
1,nice to meet you!,nice to meet you too!
2,which programming language do you use?,i like python.
3,where do you live?,i live in gwalior.
4,what is your major?,i study software engineering.


In [8]:
# df = df[(df['title'] != '[deleted]') & (df['response'] != '[deleted]')]
df.shape

(698, 2)

In [0]:
# from google.colab import files

# df.to_csv('data.csv')
# files.download('data.csv')

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

In [0]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [0]:
df['title'] = df['title'].apply(lambda x: x.lower())
df['response'] = df['response'].apply(lambda x: x.lower())

In [12]:
df['title'] = df['title'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))
df['response'] = df['response'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))

100%|██████████| 698/698 [00:00<00:00, 133932.21it/s]


In [0]:
punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

In [0]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

In [0]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [16]:
df['title'] = df['title'].progress_apply(lambda x: clean_special_chars(x, punct, punct_mapping))
df['response'] = df['response'].progress_apply(lambda x: clean_special_chars(x, punct, punct_mapping))

100%|██████████| 698/698 [00:00<00:00, 21776.60it/s]


In [0]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [0]:
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [19]:
df['title'] = df['title'].progress_apply(lambda x: correct_spelling(x, mispell_dict))
df['response'] = df['response'].progress_apply(lambda x: correct_spelling(x, mispell_dict))

100%|██████████| 698/698 [00:00<00:00, 56053.61it/s]


In [20]:
import re
# text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
df['title'] = df['title'].progress_apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
df['response'] = df['response'].progress_apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))

100%|██████████| 698/698 [00:00<00:00, 115161.05it/s]


In [21]:
df['title'] = df['title'].progress_apply(lambda x: re.sub(r'\/r\/[a-z]+ ', '', x, flags=re.MULTILINE))
df['response'] = df['response'].progress_apply(lambda x: re.sub(r'\/r\/[a-z]+ ', '', x, flags=re.MULTILINE))

100%|██████████| 698/698 [00:00<00:00, 175170.48it/s]


In [0]:
def preprocess_sentence(w):

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
#     w = re.sub(r"([?.!,¿])", r" \1 ", w)
#     w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!¿]+", " ", w)

    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
#     w = '<start> ' + w + ' <end>'
    
    return w

In [23]:
df['title'] = df['title'].progress_apply(lambda x: preprocess_sentence(x))
df['response'] = df['response'].progress_apply(lambda x: preprocess_sentence(x))

100%|██████████| 698/698 [00:00<00:00, 127078.05it/s]


In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.contrib.seq2seq import AttentionWrapper as attention_wrapper
from tensorflow.contrib.seq2seq import BeamSearchDecoder as beam_search_decoder
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.14.0


In [0]:
# title list
titles=df['title'].values
import string
title=[]
responses=df['response'].values
response=[]
for i in range(len(titles)):
  if 1 <= len(titles[i].split()) <= 15:
    if 1 <= len(responses[i].split()) <= 15:
      title.append( titles[i].lower().strip() )
      response.append( responses[i].lower().strip() )

In [26]:
print('Title samples :', len(title))
print('Response samples :', len(response))

Title samples : 698
Response samples : 698


In [27]:
import re
import pprint
from collections import Counter


# maximum length of input and target sentences including paddings
enc_sentence_length = 17
dec_sentence_length = 17

# input_batches = [
#     ['hi what is your name?', 'nice to meet you!'],
#     ['which programming language do you use?', 'see you later.'],
#     ['where do you live?', 'what is your major?'],
#     ['what do you want to drink?', 'what is your favorite beer?']]

# target_batches = [
#     ['hi this is chinmay.', 'nice to meet you too!'],
#     ['i like python.', 'bye bye.'],
#     ['i live in gwalior, madhya pradesh.', 'i study software engineering.'],
#     ['beer please!', 'leffe brown!']]


all_input_sentences = []
# for input_batch in input_batches:
#     all_input_sentences.extend(input_batches)
    
all_target_sentences = []
# for target_batch in target_batches:
#     all_target_sentences.extend(target_batches)
    
for t in range(len(title)-1):
  all_input_sentences.extend([[title[t],title[t+1]]])
  
for r in range(len(response)-1):
  all_target_sentences.extend([[response[r],response[r+1]]])
    
def tokenizer(sentence):
    tokens = re.findall(r"[\w]+|[^\s\w]", str(sentence))
    return tokens

_START_ = "_START_"
_PAD_ = "_PAD_"
_END_ = "_END_"

def build_vocab(sentences, max_vocab_size=None):
    word_counter = Counter()
    vocab = dict()
    reverse_vocab = dict()
    
    for sentence in sentences:
        tokens = tokenizer(sentence)
        word_counter.update(tokens)
        
    if max_vocab_size is None:
        max_vocab_size = len(word_counter)
    
    vocab[_START_] = 0
    vocab[_PAD_] = 1
    vocab[_END_] = 2
    vocab_idx = 3
    for key, value in word_counter.most_common(max_vocab_size):
            vocab[key] = vocab_idx
            vocab_idx += 1
            
    for key, value in vocab.items():
        reverse_vocab[value] = key
            
    return vocab, reverse_vocab, max_vocab_size

vocab,reverse_vocab,max_vocab_size = build_vocab(all_input_sentences+all_target_sentences)

def token2idx(word, vocab):
    return vocab[word]

def sent2idx(sent, vocab=vocab, max_sentence_length=enc_sentence_length, is_target=False):
    tokens = tokenizer(sent)
    current_length = len(tokens)
    pad_length = max_sentence_length - current_length
    if is_target:
        return [0] + [token2idx(token, vocab) for token in tokens] + [2] + [1] * (pad_length-1), current_length + 1
    else:
        return [token2idx(token, vocab) for token in tokens] + [1] * pad_length, current_length

def idx2token(idx, reverse_vocab):
    return reverse_vocab[idx]

def idx2sent(indices, reverse_vocab=reverse_vocab):
    return " ".join([idx2token(idx, reverse_vocab) for idx in indices])

# Enc Example
print('hi what is your name?')
print(sent2idx('hi what is your name?'))

# Dec Example
print('hi this is chinmay.')
print(sent2idx('hi this is chinmay.', max_sentence_length=dec_sentence_length, is_target=True))

idx2sent([0, 16, 41, 7, 36, 3, 2, 1, 1, 1, 1])

hi what is your name?
([240, 23, 13, 21, 238, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 6)
hi this is chinmay.
([0, 240, 51, 13, 274, 7, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 6)


"_START_ am of . people ' _END_ _PAD_ _PAD_ _PAD_ _PAD_"

In [0]:
# f = open("senttoindex.pkl","wb")
# pickle.dump(vocab,f)
# f.close()

In [0]:
# f = open("indextosent.pkl","wb")
# pickle.dump(reverse_vocab,f)
# f.close()

In [0]:
# json.load( open( "idx2sent.json" ) )
# json.dump( reverse_vocab, open( "idxtosent.json", 'w' ) )

In [0]:
# json.dump( vocab, open( "senttoindex.json", 'w' ) )

In [32]:
len(all_input_sentences)

697

In [0]:
def get_optimizer(opt):
    if opt == "adam":
        optfn = tf.train.AdamOptimizer
    elif opt == "sgd":
        optfn = tf.train.GradientDescentOptimizer
    else:
        assert(False)
    return optfn

In [0]:
def single_rnn_cell(cell_name,dim_size, train_phase = True, keep_prob = 0.75):
    if cell_name == "gru":
        cell = tf.contrib.rnn.GRUCell(dim_size)
    elif cell_name == "lstm":
        cell = tf.contrib.rnn.LSTMCell(dim_size)
    else:
        cell = tf.contrib.rnn.BasicRNNCell(dim_size)
    if train_phase and keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(
              cell=cell,
              input_keep_prob=keep_prob,
              output_keep_prob=keep_prob)
    return cell

def multi_rnn_cell(cell_name,dim_size,num_layers = 1, train_phase = True, keep_prob=0.75):
    cells = []
    for _ in range(num_layers):
        cell = single_rnn_cell(cell_name,dim_size, train_phase, keep_prob)
        cells.append(cell)
    
    if len(cells) > 1:
        final_cell = tf.contrib.rnn.MultiRNNCell(cells=cells)
    else:
        final_cell = cells[0]
    return final_cell

In [0]:
class BasicS2SModel(object):
    def __init__(self, vocab, batch_size = 2, dim_size=128, rnn_cell = 'gru', num_layers=2, max_gradient_norm=5.0, atten_size=30, 
                 learning_rate=0.001, learning_rate_decay_factor=0.98, dropout=0.2,max_inference_lenght=10,
                 max_source_len = 10, max_target_len = 10,beam_size =3, optimizer="adam", mode ='train',
                 use_beam_search = False):
        assert mode in ['train', 'inference']
        self.start_token = vocab.get(_START_)
        self.end_token = vocab.get(_END_)
        self.train_phase = True if mode == 'train' else False
        self.cell_name = rnn_cell
        self.dim_size = dim_size
        self.vocab_size = len(vocab)
        self.num_layers = num_layers
        self.keep_prob_config = 1.0 - dropout
        self.atten_size = atten_size
        
        # decoder
        self.max_inference_lenght = max_inference_lenght
        
        # beam search
        self.beam_size = beam_size
        self.beam_search = use_beam_search
        
        # learning
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.learning_rate_decay_op = self.learning_rate.assign(self.learning_rate * learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        
        # if we use beam search decoder, we need to specify the batch size and max source len
        if self.beam_search:
            self.batch_size = batch_size
            self.source_tokens = tf.placeholder(tf.int32, shape=[batch_size, max_source_len])
            self.source_length = tf.placeholder(tf.int32, shape=[batch_size,])
        else:
            self.source_tokens = tf.placeholder(tf.int32,shape=[None,None])
            self.source_length = tf.placeholder(tf.int32,shape=[None,])
            
        if self.train_phase:
            self.target_tokens = tf.placeholder(tf.int32, shape=[None, None])
            self.target_length = tf.placeholder(tf.int32, shape=[None,])
         
        with tf.variable_scope("S2S",initializer = tf.uniform_unit_scaling_initializer(1.0)):
            self.setup_embeddings()
            #self.setup_encoder()
            self.setup_bidirection_encoder()
            self.setup_attention_decoder()
                
        if self.train_phase:
            opt = get_optimizer(optimizer)(self.learning_rate)
            params = tf.trainable_variables()
            gradients = tf.gradients(self.losses, params)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
            self.gradient_norm = tf.global_norm(gradients)
            self.param_norm = tf.global_norm(params)
            self.updates = opt.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)
    
    def setup_embeddings(self):
        with tf.variable_scope("Embeddings"):
            with tf.device('/cpu:0'):
                self.enc_emd = tf.get_variable("encode_embedding", [self.vocab_size, self.dim_size])
                self.dec_emd = tf.get_variable("decode_embedding", [self.vocab_size, self.dim_size])
                self.encoder_inputs = tf.nn.embedding_lookup(self.enc_emd, self.source_tokens)
                if self.train_phase:
                    self.decoder_inputs = tf.nn.embedding_lookup(self.dec_emd, self.target_tokens)
    
    def setup_encoder(self):
        cell = multi_rnn_cell(self.cell_name,self.dim_size, self.num_layers, self.train_phase,self.keep_prob_config)
        outputs,state = tf.nn.dynamic_rnn(cell,inputs=self.encoder_inputs,sequence_length=self.source_length,dtype=tf.float32)
        self.encode_output = outputs
        self.encode_state = state
        # using the state of last layer of rnn as initial state
        self.decode_initial_state = self.encode_state[-1]
        
    def setup_bidirection_encoder(self):
        fw_cell = single_rnn_cell('gru',self.dim_size, train_phase=self.train_phase, keep_prob=self.keep_prob_config)
        bw_cell = single_rnn_cell('gru',self.dim_size, train_phase=self.train_phase, keep_prob=self.keep_prob_config)
        
        with tf.variable_scope("Encoder"):
            outputs,states = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = fw_cell,
                cell_bw = bw_cell,
                dtype = tf.float32,
                sequence_length = self.source_length,
                inputs = self.encoder_inputs
                )
            outputs_concat = tf.concat(outputs, 2)
        self.encode_output = outputs_concat
        self.encode_state = states
        
        # use Dense layer to convert bi-direction state to decoder inital state
        convert_layer = Dense(self.dim_size,dtype=tf.float32,name="bi_convert")
        self.decode_initial_state = convert_layer(tf.concat(self.encode_state,axis=1))
        
    def setup_training_decoder_layer(self):
        max_dec_len = tf.reduce_max(self.target_length, name='max_dec_len')
        training_helper = tf.contrib.seq2seq.TrainingHelper(self.decoder_inputs,self.target_length,name="training_helper")
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = self.dec_cell,
            helper = training_helper,
            initial_state = self.initial_state,
            output_layer = self.output_layer
        )
        train_dec_outputs, train_dec_last_state,_ = tf.contrib.seq2seq.dynamic_decode(
            training_decoder,
            output_time_major=False,
            impute_finished=True,
            maximum_iterations=max_dec_len)
        
        # logits: [batch_size x max_dec_len x vocab_size]
        logits = tf.identity(train_dec_outputs.rnn_output, name='logits')

        # targets: [batch_size x max_dec_len x vocab_size]
        targets = tf.slice(self.target_tokens, [0, 0], [-1, max_dec_len], 'targets')

        masks = tf.sequence_mask(self.target_length,max_dec_len,dtype=tf.float32,name="mask")
        self.losses = tf.contrib.seq2seq.sequence_loss(logits=logits,targets=targets,weights=masks,name="losses")
        
        # prediction sample for validation
        self.valid_predictions = tf.identity(train_dec_outputs.sample_id, name='valid_preds')
    
    def setup_inference_decoder_layer(self):
        start_tokens = tf.tile(tf.constant([self.start_token],dtype=tf.int32),[self.batch_size])
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding=self.dec_emd,
            start_tokens=start_tokens,
            end_token=self.end_token)
        
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = self.dec_cell,
            helper = inference_helper, 
            initial_state=self.initial_state,
            output_layer=self.output_layer)
        
        infer_dec_outputs, infer_dec_last_state,_ = tf.contrib.seq2seq.dynamic_decode(
                    inference_decoder,
                    output_time_major=False,
                    impute_finished=True,
                    maximum_iterations=self.max_inference_lenght)
        # [batch_size x dec_sentence_length], tf.int32
        self.predictions = tf.identity(infer_dec_outputs.sample_id, name='predictions')
            
    def setup_beam_search_decoder_layer(self):
        start_tokens = tf.tile(tf.constant([self.start_token],dtype=tf.int32),[self.batch_size])
        bsd = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell=self.dec_cell,
                    embedding=self.dec_emd,
                    start_tokens= start_tokens,
                    end_token=self.end_token,
                    initial_state=self.initial_state,
                    beam_width=self.beam_size,
                    output_layer=self.output_layer,
                    length_penalty_weight=0.0)
        # final_outputs are instances of FinalBeamSearchDecoderOutput
        final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(
            bsd, 
            output_time_major=False,
           # impute_finished=True,
            maximum_iterations=self.max_inference_lenght
        )
        beam_predictions = final_outputs.predicted_ids
        self.beam_predictions = tf.transpose(beam_predictions,perm=[0,2,1])
        self.beam_prob = final_outputs.beam_search_decoder_output.scores
        self.beam_ids = final_outputs.beam_search_decoder_output.predicted_ids
        
    def setup_attention_decoder(self):
        #dec_cell = multi_rnn_cell('gru',self.dim_size,num_layers=self.num_layers, train_phase = self.train_phase, keep_prob=self.keep_prob_config)
        dec_cell = [single_rnn_cell(self.cell_name,self.dim_size,self.train_phase,self.keep_prob_config) for i in range(self.num_layers)]
        if self.beam_search:
            memory = tf.contrib.seq2seq.tile_batch(self.encode_output,multiplier = self.beam_size)
            memory_sequence_length = tf.contrib.seq2seq.tile_batch(self.source_length,multiplier = self.beam_size)
        else:
            memory = self.encode_output
            memory_sequence_length = self.source_length
            
        attn_mech = tf.contrib.seq2seq.BahdanauAttention(
            num_units = self.atten_size,
            memory = memory,
            memory_sequence_length = memory_sequence_length,
            name = "BahdanauAttention"
        )
        dec_cell[0] = tf.contrib.seq2seq.AttentionWrapper(
            cell=dec_cell[0],
            attention_mechanism=attn_mech,
            attention_layer_size=self.atten_size)
        
        if self.beam_search:
            tile_state = tf.contrib.seq2seq.tile_batch(self.decode_initial_state,self.beam_size)
            initial_state = [tile_state for i in range(self.num_layers)]
            cell_state = dec_cell[0].zero_state(dtype=tf.float32,batch_size=self.batch_size*self.beam_size)
            initial_state[0] = cell_state.clone(cell_state=initial_state[0])
            self.initial_state = tuple(initial_state)
        else:
            # we use dynamic batch size
            self.batch_size = tf.shape(self.encoder_inputs)[0]
            initial_state = [self.decode_initial_state for i in range(self.num_layers)]
            cell_state = dec_cell[0].zero_state(dtype=tf.float32, batch_size = self.batch_size)
            initial_state[0] = cell_state.clone(cell_state=initial_state[0])
            self.initial_state = tuple(initial_state)
            
        print(self.initial_state)
        self.dec_cell = tf.contrib.rnn.MultiRNNCell(dec_cell)
        self.output_layer = Dense(self.vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
        if self.train_phase:
            self.setup_training_decoder_layer()
        else:
            if self.beam_search:
                self.setup_beam_search_decoder_layer()
            else:
                self.setup_inference_decoder_layer()
    
    def train_one_step(self,sess,encode_input,encode_len,decode_input,decode_len):
        feed_dict = {}
        feed_dict[self.source_tokens] = encode_input
        feed_dict[self.source_length] = encode_len
        feed_dict[self.target_tokens] = decode_input
        feed_dict[self.target_length] = decode_len
        valid_predictions,loss,_ = sess.run([self.valid_predictions,self.losses,self.updates],feed_dict=feed_dict)
        return valid_predictions,loss
    
    def inference(self,sess,encode_input,encode_len):
        feed_dict = {}
        feed_dict[self.source_tokens] = encode_input
        feed_dict[self.source_length] = encode_len
        if self.beam_search:
            predictions,probs,ids = sess.run([self.beam_predictions,self.beam_prob,self.beam_ids],feed_dict=feed_dict)
            return predictions,ids
        else:
            predictions = sess.run([self.predictions],feed_dict=feed_dict)
            return predictions
    
    def save_model(self,sess,checkpoint_dir):
        writer = tf.summary.FileWriter(checkpoint_dir, sess.graph)
        saver = tf.train.Saver(tf.global_variables())
        saver.save(sess,checkpoint_dir + "model.ckpt",global_step=self.global_step)
        
    def restore_model(self,sess,checkpoint_dir):
        saver = tf.train.Saver(tf.global_variables())
        saver.restore(sess,tf.train.latest_checkpoint(checkpoint_dir))

In [36]:
tf.reset_default_graph()
model = BasicS2SModel(vocab=vocab,mode="train")

tf.reset_default_graph()
model = BasicS2SModel(vocab=vocab,mode="inference")

print("beam")
tf.reset_default_graph()
model = BasicS2SModel(vocab=vocab,mode="inference",use_beam_search=True,beam_size=3)

W0815 10:04:41.689500 139967090608000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
W0815 10:04:41.720247 139967090608000 deprecation.py:323] From <ipython-input-34-e72657281cdd>:3: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W0815 10:04:41.730442 139967090608000 deprecation.py:323] From <ipython-input-35-5597b33b1874>:84: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bi

(AttentionWrapperState(cell_state=<tf.Tensor 'S2S/bi_convert/BiasAdd/Identity:0' shape=(?, 128) dtype=float32>, attention=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_2/Identity:0' shape=(?, 30) dtype=float32>, time=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_1:0' shape=() dtype=int32>, alignments=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros/Identity:0' shape=(?, ?) dtype=float32>, alignment_history=(), attention_state=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_3/Identity:0' shape=(?, ?) dtype=float32>), <tf.Tensor 'S2S/bi_convert/BiasAdd:0' shape=(?, 128) dtype=float32>)
(AttentionWrapperState(cell_state=<tf.Tensor 'S2S/bi_convert/BiasAdd/Identity:0' shape=(?, 128) dtype=float32>, attention=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_2/Identity:0' shape=(?, 30) dtype=float32>, time=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_1:0' shape=() dtype=int32>, alignments=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros/Identity:0' shape=(?, ?) dtype=float32>, alignment_h

W0815 10:04:49.003839 139967090608000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/seq2seq/python/ops/beam_search_decoder.py:985: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [37]:
tf.reset_default_graph()
model = BasicS2SModel(vocab=vocab,num_layers=3)

(AttentionWrapperState(cell_state=<tf.Tensor 'S2S/bi_convert/BiasAdd/Identity:0' shape=(?, 128) dtype=float32>, attention=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_2/Identity:0' shape=(?, 30) dtype=float32>, time=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_1:0' shape=() dtype=int32>, alignments=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros/Identity:0' shape=(?, ?) dtype=float32>, alignment_history=(), attention_state=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_3/Identity:0' shape=(?, ?) dtype=float32>), <tf.Tensor 'S2S/bi_convert/BiasAdd:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'S2S/bi_convert/BiasAdd:0' shape=(?, 128) dtype=float32>)


In [39]:

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess,tf.train.latest_checkpoint('./'))
    
    print("start to run")
    loss_history = []
    for epoch in range(50):
        all_preds = []
        epoch_loss = 0
#         i=0
        for input_batch, target_batch in tqdm(zip(all_input_sentences, all_target_sentences)):
            input_batch_tokens = []
            target_batch_tokens = []
            enc_sentence_lengths = []
            dec_sentence_lengths = []

            for input_sent in input_batch:
                tokens, sent_len = sent2idx(input_sent)
                input_batch_tokens.append(tokens)
                enc_sentence_lengths.append(sent_len)

            for target_sent in target_batch:
                tokens, sent_len = sent2idx(target_sent,
                             vocab=vocab,
                             max_sentence_length=dec_sentence_length,
                             is_target=True)
                target_batch_tokens.append(tokens)
                dec_sentence_lengths.append(sent_len)
#             i = i+1
#             print(i)
            batch_preds, batch_loss = model.train_one_step(sess,input_batch_tokens,enc_sentence_lengths,target_batch_tokens,dec_sentence_lengths)
            epoch_loss += batch_loss
            all_preds.append(batch_preds)
        loss_history.append(epoch_loss)
        if epoch % 1 == 0:
            print('Epoch', epoch)
            print('epoch loss: ', epoch_loss )
            for input_sent, target_sent, pred in zip(input_batch, target_batch, batch_preds):
                print('Input:', input_sent)
                print('Prediction:', idx2sent(pred, reverse_vocab=reverse_vocab))
                print('Target:', target_sent)

        
        writer = tf.summary.FileWriter('./checkpoints/', sess.graph)
        
        saver.save(sess,"./checkpoints/model.ckpt")
    # ,global_step=model.global_step


0it [00:00, ?it/s]

start to run


697it [00:50, 13.89it/s]


Epoch 0
epoch loss:  3144.470274090767
Input: my heart is broken
Prediction: _START_ move to _START_ _START_
Target: move on
Input: i broke her heart
Prediction: _START_ move on and speak
Target: go with the flow


697it [00:47, 17.79it/s]


...

Epoch 47
epoch loss:  25.000062897961698
Input: my heart is broken
Prediction: _START_ move on _START_ _START_
Target: move on
Input: i broke her heart
Prediction: _START_ go with the flow
Target: go with the flow


697it [00:47, 14.60it/s]


Epoch 48
epoch loss:  29.74657986289921
Input: my heart is broken
Prediction: _START_ move on _START_ _START_
Target: move on
Input: i broke her heart
Prediction: _START_ go with the flow
Target: go with the flow


697it [00:47, 18.11it/s]


Epoch 49
epoch loss:  24.772872344739795
Input: my heart is broken
Prediction: _START_ move on _START_ _START_
Target: move on
Input: i broke her heart
Prediction: _START_ go with the flow
Target: go with the flow


In [0]:
# from google.colab import files
# files.download("/content/file.zip")

In [43]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = BasicS2SModel(vocab=vocab,mode="inference",use_beam_search=False,num_layers=3)
    saver = tf.train.Saver(tf.global_variables())
    saver.restore(sess,tf.train.latest_checkpoint('./checkpoints'))
    
    
    for input_batch, target_batch in zip(all_input_sentences[:40], all_target_sentences[:40]):
      batch_preds = []
      batch_tokens = []
      batch_sent_lens = []
      for input_sent in input_batch:
          tokens, sent_len = sent2idx(input_sent)
          batch_tokens.append(tokens)
          batch_sent_lens.append(sent_len)

      batch_preds = model.inference(sess,batch_tokens,batch_sent_lens)
      # print(batch_preds)
      
      # ans = []
      for preds in batch_preds[0]:
      #     # print(preds)
      #     ans1 = []
          for i in range(len(preds)-1):
            if preds[i] == preds[i+1]:
      #         if preds[i] not in ans1:
      #           ans1.append(preds[i])
      #           if i == len(preds)-1:
      #             ans1.append(preds[i+1])
              preds[i+1:9] = 1
              preds[9] = 2
      #     ans.append(ans1)
      # batch_preds[0]
      for input_sent, target_sent, pred in zip(input_batch, target_batch, batch_preds[0]):
          print('Input:', input_sent)
          print(pred)
          print('Prediction:', idx2sent(pred, reverse_vocab=reverse_vocab))
          print('Target:', target_sent)


(AttentionWrapperState(cell_state=<tf.Tensor 'S2S/bi_convert/BiasAdd/Identity:0' shape=(?, 128) dtype=float32>, attention=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_2/Identity:0' shape=(?, 30) dtype=float32>, time=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_1:0' shape=() dtype=int32>, alignments=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros/Identity:0' shape=(?, ?) dtype=float32>, alignment_history=(), attention_state=<tf.Tensor 'S2S/AttentionWrapperZeroState/zeros_3/Identity:0' shape=(?, ?) dtype=float32>), <tf.Tensor 'S2S/bi_convert/BiasAdd:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'S2S/bi_convert/BiasAdd:0' shape=(?, 128) dtype=float32>)
Input: hi what is your name ?
[  0 240   1   1   1   1   1   1   1   2]
Prediction: _START_ hi _PAD_ _PAD_ _PAD_ _PAD_ _PAD_ _PAD_ _PAD_ _END_
Target: hi this is sofia .
Input: nice to meet you
[  0 241  97 356 300 821   1   1   1   2]
Prediction: _START_ nice something delicious birthday books _PAD_ _PAD_ _PAD_ _END_
Target: nice to m